Building a multi-layer perceptron with one hidden layer from scratch and test it on MNIST data

In [124]:
import torch

# Activation function

In [125]:
def sigma(x):
    return torch.tanh(x)

In [126]:
def dsigma(x):
    return 1 - sigma(x)

# Loss

In [127]:
def loss(v, t):
    return (v-t).pow(2).sum()

In [128]:
def dloss(v, t):
    return 2*(v-t)

# Importing the data

In [129]:
import dlc_practical_prologue as prologue

train_input, train_target, test_input,test_target = prologue.load_data(one_hot_labels = True,
                                                                      normalize = True)

train_target = train_target * 0.9
test_target  = test_target * 0.9
print(train_input.shape)

nb_hidden = 50
epsilon = 1e-6
nb_classes = train_target.size(1)
nb_train_samples = train_input.size(0)
learning_rate = 0.1
step = learning_rate/nb_train_samples

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
torch.Size([1000, 784])


In [130]:
x = train_input

In [146]:
w1 = torch.empty(nb_hidden, train_input.size(1)).normal_(0, epsilon)
b1 = torch.empty(nb_hidden, train_input.size(0)).normal_(0, epsilon)
w2 = torch.empty(nb_classes, nb_hidden).normal_(0, epsilon)
b2 = torch.empty(nb_classes, train_input.size(0)).normal_(0, epsilon)


dl_dw1 = torch.empty(w1.size())
dl_db1 = torch.empty(b1.size())
dl_dw2 = torch.empty(w2.size())
dl_db2 = torch.empty(b2.size())

In [168]:
w2.shape

torch.Size([10, 50])

In [169]:
def forward_pass(w1, w2, b1, b2, x):
    s1 = torch.add(torch.mm(w1, torch.transpose(x, 0, 1)), b1)
    x1 = sigma(s1)
    s2 = torch.add(torch.mm(w2,x1), b2)
    x2 = sigma(s1)
    
    return s1, x1, s2, x2

In [170]:
#b1.shape
#s1 = torch.add(torch.mm(w1, torch.transpose(x, 0, 1)), b1)
#x1 = sigma(s1)

#x1.shape

torch.Size([50, 1000])

In [174]:
#forward_pass(w1, w2, b1, b2, x)

In [85]:
def backward_pass(w1, b1, w2, b2,
                  t,
                  x, s1, x1, s2, x2,
                 dl_dw1, dl_db1, dl_dw2, dl_db2):
    
    dl_dx2 = loss(x2, t)
    dl_ds2 = torch.mm(dl_dx2, dsigma)
    dl_dx1 = torch.mm(dl_s2, w2)
    dl_ds1 = torch.mm(dl_dx1, dsigma(s1))
    
    dl_dw2 = torch.mm(dl_ds2, x1)
    dl_db2 = torch.mm(dl_ds2, 1)
    dl_db1 = torch.mm(dl_ds1, 1)
    dl_dw1 = torch.mm(dl_ds1, x)

In [86]:
for k in range(1000):
    
    """
    Performing 1,000 gradient steps with a step size equal to 0.1 divided
    by the number of training samples(variable--steps)
    First reset the tensors to zero for summing up the gradients and doing
    a forward and backward pass for each training example
    """
    dl_dw1 = torch.zeros(dl_dw1.shape)
    dl_db1 = torch.zeros(dl_db1.shape)
    dl_dw2 = torch.zeros(dl_dw2.shape)
    dl_db2 = torch.zeros(dl_db2.shape)
    
    for n in range(nb_train_samples):
        #forward prop
        
        x, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, train_input[n])
    
        #backward prop
        backward_pass(w1, b1, w2, b2,
                     train_input[n],
                     x, s1, x1, s2, x2,
                     dl_w1, dl_db1, dl_dw2, dl_db2)
    
        #update rule
        w1 = w1 - step * dl_dw1
        b1 = b1 - step * dl_db1
        w2 = w2 - step * dl_dw2
        w1 = w1 - step * dl_db2
    
    
    

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)